<a href="https://colab.research.google.com/github/byunsy/equal-weight-index-fund/blob/main/Equal_Weight_S%26P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equal-Weight S&P 500 Index Fund

This notebook is quite a short study on how to calculate how many shares of each S&P 500 constituent we should purchase to get an equally weighted index fund. 

## Import Necessary Modules

In [3]:
import numpy as np 
import pandas as pd 
import requests 
import math 

In [4]:
!pip install -q finance-datareader
import FinanceDataReader as fdr

In [ ]:
from google.colab import files
uploaded = files.upload()

## Attain S&P Stock Listing

Get a list of all the companies in the S&P 500. 

In [26]:
sp500 = pd.read_csv('sp_500_stocks.csv')
sp500

,Symbol
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## API Call

We first need to have a test api token to use IEX Cloud APIs (This will remain private). You can receive sandbox Text APIs from the IEX Cloud API website. 

In [ ]:
from iex_api import IEX_CLOUD_API_TOKEN

In [27]:
# To take an example of what we get from IEX Cloud, we will take Microsoft
symbol='MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
ms_data = requests.get(api_url).json()

ms_data

{'avgTotalVolume': 26439018,
 'calculationPrice': 'close',
 'change': 0.63,
 'changePercent': 0.00284,
 'close': None,
 'closeSource': 'oaliciff',
 'closeTime': None,
 'companyName': 'Microsoft Corporation',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 231.06,
 'iexCloseTime': 1674193133784,
 'iexLastUpdated': 1676207857158,
 'iexMarketPercent': 0.0025636064874515084,
 'iexOpen': 235.38,
 'iexOpenTime': 1680901496278,
 'iexRealtimePrice': 228.82,
 'iexRealtimeSize': 35,
 'iexVolume': 901,
 'isUSMarketOpen': False,
 'lastTradeTime': 1613298155475,
 'latestPrice': 234.11,
 'latestSource': 'Close',
 'latestTime': 'January 21, 2021',
 'latestUpdate': 1641572971687,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,

We can now get specific information about our data using indices. 

In [28]:
print("LATEST PRICE:", ms_data['latestPrice'])
print("MARKET CAPITALIZATION:", ms_data['marketCap'])

LATEST PRICE: 234.11
MARKET CAPITALIZATION: 1729153670698


**NOTE:**

Since we are using sandbox test APIs, the values returned are not real. 

## Data Preprocessing


We will now transfer our attained data into a data frame. 

In [29]:
df_columns = ['Symbol', 'Latest Price','Market Capitalization', 'Number Of Shares to Purchase']
df = pd.DataFrame(columns=df_columns)

for symbol in sp500['Symbol']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append(pd.Series([symbol, data['latestPrice'], 
                              data['marketCap'], 'N/A'], index=df_columns), 
                              ignore_index = True)
df   

,Symbol,Latest Price,Market Capitalization,Number Of Shares to Purchase
0,A,132,39234353019,N/A
1,AAL,16.3,10015910065,N/A
2,AAP,172,11598099591,N/A
3,AAPL,142.1,2404888952813,N/A
4,ABBV,115.51,200925169770,N/A
...,...,...,...,...
500,YUM,107.74,33093214402,N/A
501,ZBH,169.417,34588572110,N/A
502,ZBRA,413.27,22582138332,N/A
503,ZION,47.68,8160254259,N/A


However, we immediately realize this takes quite a long time to process. This is because API calls can be time-consuming. One way to cope with this is to use batch API calls.

In [38]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [42]:
symbol_batch = list(chunks(sp500['Symbol'], 100))
symbol_strings = []

for batch in symbol_batch:
    symbol_strings.append(','.join(batch))

df = pd.DataFrame(columns=df_columns)

for symbol_string in symbol_strings:

    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol, data[symbol]['quote']['latestPrice'], 
                                  data[symbol]['quote']['marketCap'], 'N/A'], 
                                  index=df_columns), ignore_index = True)
        
df

,Symbol,Latest Price,Market Capitalization,Number Of Shares to Purchase
0,A,128.870,39141736111,N/A
1,AAL,15.672,9746614390,N/A
2,AAP,168.330,11392472602,N/A
3,AAPL,137.420,2394126337201,N/A
4,ABBV,111.690,203452084840,N/A
...,...,...,...,...
500,YUM,107.870,32497982801,N/A
501,ZBH,169.130,34339924918,N/A
502,ZBRA,415.400,22170426877,N/A
503,ZION,49.433,8138586140,N/A


## Calculate the Number of Shares to Purchase

Given a value of our portfolio, we can now calculate the number of shares of each constituent to purchase.




In [60]:
PORTFOLIO_SIZE = 100000

position_size = float(PORTFOLIO_SIZE) / len(df.index)
print(position_size)

198.01980198019803


In [61]:
position_size = float(PORTFOLIO_SIZE) / len(df.index)

for i in range(len(df['Symbol'])):
    df.loc[i, 'Number Of Shares to Purchase'] = math.floor(position_size / df['Latest Price'][i])

df

,Symbol,Latest Price,Market Capitalization,Number Of Shares to Purchase
0,A,128.870,39141736111,1
1,AAL,15.672,9746614390,12
2,AAP,168.330,11392472602,1
3,AAPL,137.420,2394126337201,1
4,ABBV,111.690,203452084840,1
...,...,...,...,...
500,YUM,107.870,32497982801,1
501,ZBH,169.130,34339924918,1
502,ZBRA,415.400,22170426877,0
503,ZION,49.433,8138586140,4
